In [8]:
from paramiko import SSHClient, AutoAddPolicy
import os
import subprocess

In [9]:
class BackendManager:
    def __init__(self, server, uname, app):
        self.server = server
        self.uname = uname
        self.app = app
        self.client = SSHClient()
        self.client.set_missing_host_key_policy(AutoAddPolicy())
        self.connected = False
        
        
    def connect(self):
        self.client.connect(self.server, username=self.uname)
        self.connected = True
        
    def close(self):
        self.client.close()
        self.connected = False
        
    def generate_experiments(self, expression_dir, args):
        if self.connected:
            cmd = "source ~/.analyzer; "
            cmd += "cd {}; ".format(expression_dir)
            cmd += "python generate-variants-linnea.py {};".format(" ".join(args))
            
            _, stdout, _ = self.client.exec_command(cmd)
            
            ret = stdout.readlines()
            if "Generated Variants" in ret[-1]:
                return "Generated Variants on Cluster."
            else:
                return "Cound not generate variants."
        else:
            return -1
        
        
    def check_if_file_exists(self, file_path):
        if self.connected:
            cmd = "test -f {};".format(file_path)
            _, stdout, _ = self.client.exec_command(cmd)
            ret = stdout.channel.recv_exit_status()
            if stdout.channel.recv_exit_status() == 0:
                return True
            return False
        return -1
        
    def run_experiments(self, runner_file):
        if self.connected:
            args_dir, script = os.path.split(runner_file)
            
            cmd = "source ~/.analyzer; "
            cmd += "cd {}; ".format(args_dir)
            cmd += "{} {};".format(self.app, script)
            
            _, stdout, _ = self.client.exec_command(cmd)
            #call =  'ssh -l as641651 login18-1.hpc.itc.rwth-aachen.de'.split()
            #ret = subprocess.check_output(call + [cmd,])
           # ret = stdout.readlines()
           # print(ret)
            if stdout.channel.recv_exit_status() == 0:
                return True
            print(stdout.channel.recv_exit_status())
            return False
        return -1
    
    def check_slrum_status(self):
        if self.connected:
            cmd = "squeue -u {}".format(self.uname)
            _, stdout, _ = self.client.exec_command(cmd)
            return stdout.readlines()
        
    def copy_file_from_backend(self,src_path, target_path):
        pass
            
         

In [10]:
app = "/home/as641651/Linnea/src/julia-e5-2650-intel19/julia"
bm = BackendManager("login18-1.hpc.itc.rwth-aachen.de", "as641651", app)

In [11]:
bm.connect()

In [12]:
bm.check_slrum_status()

['             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)\n']

In [13]:
expression_dir = "~/PhD/performance-analyazer/Experiment2/Random1/variants-linnea/"

In [176]:
bm.generate_experiments(expression_dir, ['100', '100', '112'])

'Generated Variants on Cluster.'

In [7]:
file_path = os.path.join(expression_dir,
                        "experiments",
                        "100_100_112",
                        "runner.jl")

bm.check_if_file_exists(file_path)

True

In [211]:
bm.run_experiments(file_path)

True

In [212]:
run_times_path = os.path.join(expression_dir,
                        "experiments",
                        "100_100_112",
                        "run_times.txt")

bm.check_if_file_exists(run_times_path)

True

In [213]:
bm.close()